In [ ]:
# set this to point to the parameter file used
const PARAMETER_FILE = "model.parameters.json"
# set this to point to the .results file containing the simulations
const RESULTS_PREFIX = "Powder"#.results, .high.results etc
const WEATHER_FILE   = "data/TGA.daily.df.csv"
const LOW_STOCKING_RESULTS = "Powder.results"

In [ ]:
using JSON, SDDP,DataFrames#, StatPlots
data = JSON.parsefile(PARAMETER_FILE)
results = SDDP.load(string(RESULTS_PREFIX, ".results"));
low = SDDP.load(string(RESULTS_PREFIX, ".low.results"));
high = SDDP.load(string(RESULTS_PREFIX, ".high.results"));
low_stocking = SDDP.load(LOW_STOCKING_RESULTS);

In [ ]:
using Plots, StatPlots
const mm = Plots.mm
const pt = Plots.pt
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

In [ ]:
# Plot Figure 1 in the paper
plot(size=(750,500), left_margin=5mm, bottom_margin=5mm)
milking_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_milking"] + data["energy_for_maintenance"]
dry_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_dry"] + data["energy_for_maintenance"]
plot!(milking_requirements, label="Total: Lactating Cow", linewidth=3, color="#00467F")
plot!(dry_requirements, label="Total: Dry Cow", linewidth=3, color="#00467F", linestyle=:dot)
plot!(data["energy_for_bcs_milking"],label="Change in BCS: Lactating Cow", linewidth=3, color="#e65100")
plot!(data["energy_for_bcs_dry"],label="Change in BCS: Lactating Cow", linewidth=3, color="#e65100", linestyle=:dot)
hline!([data["energy_for_maintenance"]],label="Maintenance", linewidth=3, color="#009AC7", linestyle=:solid)
plot!(data["energy_for_pregnancy"], label="Pregnancy", linewidth=3, color="#009AC7", linestyle=:dash)
plot!(ylims=(-100,900), xlabel="Weeks since calving", ylabel="Energy Requirement (MJ/Week)", legend=:topleft)
savefig("energy.pdf")

In [ ]:
# Plot Figure 4
hgh = 3
plot(xlabel="Quantity of Palm Kernel Fed (kg/Cow/Day)", ylabel="Penalty (\$/Cow/Day)",legend=false,ylims=(0,1.25))
xbx = [0.0, 2.0, 2.0, 0.0]
ybx = [0, 0, 2hgh, 2hgh]
plot!(Plots.Shape(xbx, ybx), fillalpha=0.5, w=0, c="green", alpha=0)
plot!(Plots.Shape(2+xbx, ybx), fillalpha=0.5, w=0, c="orange", alpha=0)
plot!(Plots.Shape(4+xbx, ybx), fillalpha=0.5, w=0, c="red", alpha=0)
plot!([0, 3, 4, 5, 6], [0, 0, 0.25, 0.75,1.75], w=5, c="#00467F")
plot!(size=(4 * 175,2 * 175), left_margin=5mm, bottom_margin=5mm)
annotate!([(1,0.4,text("Fat Evaluation Index\nGrade A")), (3,0.4,text("Fat Evaluation Index\nGrade B")), (5,0.4,text("Fat Evaluation Index\nGrade C"))])
savefig("fei_penalty.pdf")

In [ ]:
# Plot Figure 5
df = readtable(WEATHER_FILE)
q = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
rainfall = unstack(by(df, :week) do io
   DataFrame(
        rainfall = quantile(io[:rainfall], q),
        quantile = q
        ) 
        end, :quantile, :rainfall)
plot(rainfall, Symbol(0.5), w=3, c="#00467F")
plot!(rainfall, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
rainfall_plot = plot!(legend=false, title="(a)", xlabel="Week of Year", ylabel="Rainfall\n(mm/Week)")
evapotranspiration = unstack(by(df, :week) do io
   DataFrame(
        evapotranspiration = quantile(io[:evapotranspiration], q),
        quantile = q
        ) 
        end, :quantile, :evapotranspiration)
plot(evapotranspiration, Symbol(0.5), w=3, c="#00467F")
plot!(evapotranspiration, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
evapotranspiration_plot = plot!(legend=false, title="(b)", xlabel="Week of Year", ylabel="Evapotranspiration\n(mm/Week)")
plot(rainfall_plot, evapotranspiration_plot, layout=(1,2), size=(1000,375), left_margin=8mm, bottom_margin=5mm)
savefig("weather.pdf")

In [ ]:
# Plot Figures 5 and 6
function getresults(results, key, scalefactor=1.0)
    Q = [0.0, 0.05, 0.5, 0.95, 1.0]
    g2 = hcat([scalefactor*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5])
end
function plotpowder(islow, low, high, key, ylims, ylabel,title,scalefactor=1.0)
    wks = 24:52
    blue = getresults(low, key, scalefactor)[wks, :]
    orange = getresults(high, key, scalefactor)[wks, :]

    gray = getresults(vcat(low, high), key, scalefactor)[1:24, :]
    
    plot()
    plot!(gray, :week, :x10, fill=(:x90, "gray"), c="gray", fillalpha=0.3, alpha=0.0)
    plot!(gray, :week, :x25, fill=(:x75, "gray"), c="gray", fillalpha=0.3, alpha=0.0)
    plot!(gray, :week, :x50, c="gray", w=2, linestyle=:dot)
    
    if islow
        plot!(blue,   :week, :x10, fill=(:x90, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
        plot!(blue,   :week, :x25, fill=(:x75, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
        plot!(blue, :week, :x50, c="#00467F", w=2, linestyle=:dot)
    else
        plot!(orange, :week, :x10, fill=(:x90, "#e65100"), c="#e65100", fillalpha=0.3, alpha=0)
        plot!(orange, :week, :x25, fill=(:x75, "#e65100"), c="#e65100", fillalpha=0.3, alpha=0)
        plot!(orange, :week, :x50, c="#e65100", w=2, linestyle=:dot)
    end
    plot!(1:24, scalefactor*high[10][key][1:24], color="gray", linewidth=3)
    if islow
        plot!(wks, scalefactor*low[10][key][wks], color="#00467F", linewidth=3)
    else
        plot!(wks, scalefactor*high[10][key][wks], color="#e65100", linewidth=3)
    end
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,52), xticks=(xticks, xticklabels), xlabel="")
end

function plotall(islow, low, high)
    plot(
        plotpowder(islow, low, high, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)"),
        plotpowder(islow, low, high, :P, (1.25,3.25), "Pasture Cover\n(t/Ha)","(b)", 0.001),
        plotpowder(islow, low, high, :W, (0,150), "Soil Moisture\n(mm)","(c)"),
        plotpowder(islow, low, high, :gr, (0,70), "Pasture Growth\n(kg/Day)", "(d)"),
        plotpowder(islow, low, high, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(e)", 1 / 3 / 7),
        plotpowder(islow, low, high, :milk, (0,2.5), "Milk Production\n(kg/Cow/Day)", "(f)", 1 / 3 / 7),
        layout=(2,3), size=(1500,600)
    )
end
plotall(false, low, high)
savefig("farm_high.pdf")
plotall(true, low, high)
savefig("farm_low.pdf")

In [ ]:
# Plot Figure 7
fixedcost = data["fixed_cost"]
density([r[:objective]-fixedcost for r in low_stocking], label="2.7 Cows/Ha", color="#00467F", linewidth=3)
density!([r[:objective]-fixedcost for r in results], label="3.0 Cows/Ha", color="#e65100", linewidth=3, linestyle=:dot)
vline!([mean([r[:objective]-fixedcost for r in low_stocking])], label="", color="#00467F", linewidth=3)
vline!([mean([r[:objective]-fixedcost for r in results])], label="", color="#e65100", linewidth=3, linestyle=:dot)
plot!(size=(4 * 150,3 * 150), xlabel="Operating Profit (\$/Ha)\n", xlims=(-4000, 8000), ylims=(0, 0.00035), ylabel="Fraction of Simulations")
plot!(legend=:topleft, bottom_margin=7.0Plots.mm, right_margin=5Plots.mm)
savefig("density.pdf")